In [ ]:
!pip install contractions
!pip install nltk
!pip install textblob

**Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#import googletrans
#from googletrans import Translator

# yazim degistirme you've => you have
import contractions
# regex
import re

from textblob import TextBlob
from textblob import Word

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download("all")

**Create Dataset**

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/complaint_analysis_nlp_caretta/consumer_complaints.csv')

dataset.drop(["date_received", "sub_product", "issue", "sub_issue", "company_public_response",
              "company", "state", "zipcode", "tags", "consumer_consent_provided", "submitted_via",
              "date_sent_to_company", "company_response_to_consumer", "timely_response", "consumer_disputed?",
              "complaint_id"], axis = 1, inplace = True)

dataset.dropna(inplace=True)

dataset.drop_duplicates(keep = False, inplace = True)

print(dataset[dataset['product'] == "Debt collection"].count())
print(dataset[dataset['product'] == "Mortgage"].count())
print(dataset[dataset['product'] == "Credit card"].count())
print(dataset[dataset['product'] == "Credit reporting"].count())
print(dataset[dataset['product'] == "Bank account or service"].count())

print(dataset.info())
print(dataset)

**Translate Dataset**

In [ ]:
def translator(text, sourceLanguage, targetLanguage):
  translator = Translator()
  result = translator.translate(text, src=sourceLanguage, dest=targetLanguage)

  return result.text

def translatedCSV(data):
  size = len(data)

  for i in range(0,size):
    dataset["consumer_complaint_narrative"].iloc[i] = translator(dataset["consumer_complaint_narrative"].iloc[i], "en", "tr")

    #print(dataset["consumer_complaint_narrative"].iloc[i])
    #print("\n")

# not using
#translatedCSV(dataset)


**Text Preprocessing**

In [ ]:
# product distribution show
#fig,ax = plt.subplots(figsize=(18,6))
#sns.countplot(x='product',data=dataset)

drop_class_index_list = [dataset[dataset["product"] == "Consumer Loan"].index,
                         dataset[dataset["product"] == "Student loan"].index,
                         dataset[dataset["product"] == "Payday loan"].index,
                         dataset[dataset["product"] == "Money transfers"].index,
                         dataset[dataset["product"] == "Other financial service"].index,
                         dataset[dataset["product"] == "Prepaid card"].index]

for drop_class_index in drop_class_index_list:
  dataset.drop(drop_class_index, axis=0, inplace=True)

# product distribution show
#fig,ax = plt.subplots(figsize=(18,6))
#sns.countplot(x='product',data=dataset)

# dataset index reset
#dataset = dataset.reset_index()

# convert to lowercase
#dataset['consumer_complaint_narrative'] = dataset['consumer_complaint_narrative'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))

# expand constractions
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

def decontracted_dataset(data,column_name):
  for i in data[column_name]:
    dataset[column_name] = dataset[column_name].replace([i],decontracted(i))

# not working expand constractions
#decontracted_dataset(dataset,"consumer_complaint_narrative")
#print(dataset.head(100).to_string())

# feature selection
#dataset.drop(["level_0","index"], axis = 1, inplace = True)

# Correcting Spelling
#dataset['consumer_complaint_narrative'] = dataset['consumer_complaint_narrative'].apply(lambda x: str(TextBlob(x).correct()))

print(dataset.head(100).to_string())
print("\n\n")

def lemmatization(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

# Lemmatization (köklerine indirgeme)##
#dataset["consumer_complaint_narrative"] = dataset["consumer_complaint_narrative"].apply(lemmatization)

def preprocess(df, column):
    df[column] = df[column].str.lower()  # converting to lower case

    decontracted_dataset(df, column) # expand constractions

    stop = stopwords.words('english')
    df[column] = df[column].apply(lambda x: ' '.join([i for i in x.split() if i not in stop])) # drop stopwords

    df[column] = df[column].str.replace('/^#\w+$/', '')  # removing hashtags
    df[column] = df[column].str.replace('[^\w\s]', '') # removing punctuation
    df[column] = df[column].str.replace('[_]', '') 
    df[column] = df[column].str.replace('\d', '')  # removing numbers
    df[column] = df[column].str.replace('\s\s+', ' ')  # converting multiple spaces to single space

    #df[column] = df[column].apply(lambda x: str(TextBlob(x).correct())) # Correcting Spelling (yazım düzeltme)

    df[column] = df[column].apply(lemmatization) # Lemmatization (köklerine indirgeme)

    return df

dataset = preprocess(dataset, "consumer_complaint_narrative")
print(dataset.head(100).to_string())




**Oversampling-Undersampling**

In [ ]:
import pandas as pd

# for oversampling and undersampling
from sklearn.utils import resample

# for train and test
from sklearn.model_selection import train_test_split

# verilerin dağılımı
print("\nVerilerin Dağılımı : \n")
index = pd.Index(dataset["product"])
print(index.value_counts())
print("\n\n\n\n")

#veri seti dengeleştirme

# sentetik veri ekleme
def oversampling(minority, majority, is_replace, random_state):
  return resample(minority, n_samples=majority, replace=is_replace, random_state=random_state) 

# veri azaltma
def undersampling(majority, minority, is_replace, random_state):
  return resample(majority, n_samples=minority, replace=is_replace, random_state=random_state)

dataset_debt_collection = dataset[dataset["product"] == "Debt collection"]
dataset_mortgage = dataset[dataset["product"] == "Mortgage"]
dataset_credit_reporting = dataset[dataset["product"] == "Credit reporting"]
dataset_credit_card = dataset[dataset["product"] == "Credit card"]
dataset_bank_account_or_service = dataset[dataset["product"] == "Bank account or service"]

# using undersampling
dataset_debt_collection_downsampled = undersampling(dataset_debt_collection, 10000, False, 1234)
dataset_mortgage_downsampled = undersampling(dataset_mortgage, 10000, False, 1234)
dataset_credit_reporting_downsampled = undersampling(dataset_credit_reporting, 10000, False, 1234)

# using oversampling
dataset_credit_card_upsampled = oversampling(dataset_credit_card, 8500, True, 1234)
dataset_bank_account_or_service_upsampled = oversampling(dataset_bank_account_or_service, 6500, True, 1234)

# concat undersampling dataset and oversampling dataset
dataset_upsampled_downsampled = pd.concat([dataset_debt_collection_downsampled, dataset_mortgage_downsampled, 
                                dataset_credit_reporting_downsampled, dataset_credit_card_upsampled,
                                dataset_bank_account_or_service_upsampled], axis=0)

#shuffle
dataset_upsampled_downsampled = dataset_upsampled_downsampled.sample(frac=1)

# oversampling ve undersampling işlemleri sonrası verilerin dağılımı
print("\nOversampling ve Undersampling İşlemleri Sonrası Verilerin Dağılımı : \n")
index = pd.Index(dataset_upsampled_downsampled["product"])
print(index.value_counts())
print("\n\n\n\n")

# split x and y
y = dataset_upsampled_downsampled["product"]
x = dataset_upsampled_downsampled["consumer_complaint_narrative"]

print("x shape: ", x.shape)
print("y shape: ", y.shape)

# split train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print(y_train)




Verilerin Dağılımı : 

Debt collection            16860
Mortgage                   14902
Credit reporting           11511
Credit card                 7895
Bank account or service     5671
Name: product, dtype: int64






Oversampling ve Undersampling İşlemleri Sonrası Verilerin Dağılımı : 

Mortgage                   10000
Credit reporting           10000
Debt collection            10000
Credit card                 8500
Bank account or service     6500
Name: product, dtype: int64





x shape:  (45000,)
y shape:  (45000,)
X_train:  (33750,)
X_test:  (11250,)
y_train:  (33750,)
y_test:  (11250,)
519175    Bank account or service
209052                   Mortgage
255366                   Mortgage
240178                   Mortgage
527932                   Mortgage
                   ...           
290829                Credit card
490619                   Mortgage
297336                Credit card
493414            Debt collection
506912                Credit card
Name: product, Length:

**Label Encoding and TF-IDF Vectorization**

In [ ]:
from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer

# label encoding for product column
enc = preprocessing.LabelEncoder()
y_train_label_encoder = enc.fit_transform(y_train)
y_test_label_encoder = enc.fit_transform(y_test)

# tf-idf for x_train and x_test
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
#tfidf_vect.fit(dataset['consumer_complaint_narrative'])

#X_train_tfidf =  tfidf_vect.fit_transform(X_train)
#X_test_tfidf =  tfidf_vect.fit_transform(X_test)

fitted_vectorizer = tfidf_vect.fit(X_train)
X_train_tfidf = fitted_vectorizer.transform(X_train)
X_test_tfidf = fitted_vectorizer.transform(X_test)

print("X_train_tfidf:", X_train_tfidf)
print("X_test_tfidf:", X_test_tfidf)




X_train_tfidf:   (0, 4952)	0.06809310560609141
  (0, 4943)	0.09247139105468151
  (0, 4913)	0.0922814934484676
  (0, 4849)	0.08715790250187638
  (0, 4818)	0.16949298149086045
  (0, 4756)	0.182606320665604
  (0, 4067)	0.14341203779594855
  (0, 3719)	0.17267687232976675
  (0, 3506)	0.1121749886490924
  (0, 3361)	0.11884103746830554
  (0, 3258)	0.3396358203811175
  (0, 3018)	0.08324931916324199
  (0, 2897)	0.08996135833135899
  (0, 2848)	0.17531668412593018
  (0, 2528)	0.5603506941260252
  (0, 2406)	0.09260888925920012
  (0, 2367)	0.0904406458387793
  (0, 2299)	0.16383409752440944
  (0, 2046)	0.11914455819963152
  (0, 1905)	0.0758423948353096
  (0, 1669)	0.13520016677422897
  (0, 1273)	0.15986236950990798
  (0, 1030)	0.2394270919764234
  (0, 926)	0.07208637660463477
  (0, 775)	0.17611712409044375
  :	:
  (33749, 1280)	0.55210118340925
  (33749, 1181)	0.1824960498426305
  (33749, 1101)	0.19273125617133505
  (33749, 1030)	0.04057836343710549
  (33749, 999)	0.08417694422421247
  (33749, 995)	

**Machine Learning Algorithm's**

In [ ]:
from sklearn import model_selection, preprocessing, metrics
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

#FITTING THE CLASSIFICATION MODEL using Logistic Regression (tf-idf)
lr_tfidf = LogisticRegression(solver = 'liblinear', C = 10, penalty = 'l2')
lr_tfidf.fit(X_train_tfidf, y_train)  

#Predict y value for test dataset
y_predict_LR = lr_tfidf.predict(X_test_tfidf)

x_predict_LR = lr_tfidf.predict(X_train_tfidf)

# Logistic predictions performance
print("Logistic Regression test accuracy: \n")
print(classification_report(y_test, y_predict_LR))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(lr_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("Logistic Reg mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

#FITTING THE CLASSIFICATION MODEL using SVM (tf-idf)
svm_tfidf = SVC(kernel = "rbf", C = 1)
svm_tfidf.fit(X_train_tfidf, y_train)

#Predict y value for test dataset
y_predict_SVM = svm_tfidf.predict(X_test_tfidf)

#x_predict_SVM = svm_tfidf.predict(X_train_tfidf)

# SVM predictions performans
print("SVM Test accuracy: \n")
print(classification_report(y_test, y_predict_SVM))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(svm_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("SVM mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, eta=0.3, silent=1, subsample=0.8)
xgb_model.fit(X_train_tfidf, y_train)

xgb_predict = xgb_model.predict(X_test_tfidf)

print("\n---------------------------------------------\n")

"""scores = cross_val_score(xgb_model, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("XGBoost mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

print("XGBoost Test accuracy: \n")
print(classification_report(y_test, xgb_predict))

# #FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)  

# #Predict y value for test dataset
y_predict_nb = nb_tfidf.predict(X_test_tfidf)

x_predict_nb = nb_tfidf.predict(X_train_tfidf)

# # naive bayes predictions performance
print("Naive Bayes test accuracy: \n")
print(classification_report(y_test, y_predict_nb))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(nb_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("NB mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

# #FITTING THE CLASSIFICATION MODEL using Random Forest(tf-idf)
random_forrest_tfidf = RandomForestClassifier()
random_forrest_tfidf.fit(X_train_tfidf, y_train)

y_predict_random_forest = random_forrest_tfidf.predict(X_test_tfidf)

x_predict_random_forest = random_forrest_tfidf.predict(X_train_tfidf)

# # random forest predictions performance
print("Random Forest test accuracy: \n")
print(classification_report(y_test, y_predict_random_forest))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(random_forrest_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("Random Forest mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

# #FITTING THE CLASSIFICATION MODEL using Decision Tree(tf-idf)
decision_tree_tfidf =  tree.DecisionTreeClassifier()
decision_tree_tfidf.fit(X_train_tfidf, y_train)

y_predict_decision_tree = decision_tree_tfidf.predict(X_test_tfidf)

x_predict_decision_tree = decision_tree_tfidf.predict(X_train_tfidf)

# # decision tree predictions performance
print("Decision Tree test accuracy: \n")
print(classification_report(y_test, y_predict_decision_tree))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(decision_tree_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("Decision Tree mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

# #FITTING THE CLASSIFICATION MODEL using Gaussian Naive Bayes(tf-idf)
gaussian_nb_tfidf = GaussianNB()
gaussian_nb_tfidf.fit(X_train_tfidf.todense(), y_train)  

y_predict_gaussian_nb = gaussian_nb_tfidf.predict(X_test_tfidf.todense())

# # gaussian naive bayes predictions performance
print("Gaussian Naive Bayes: \n")
print(classification_report(y_test, y_predict_gaussian_nb))

"""scores = cross_val_score(gaussian_nb_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("Gaussian Naive Bayes mean: ", scores.mean())"""

print("\n---------------------------------------------\n")

# #FITTING THE CLASSIFICATION MODEL using KNN(tf-idf)
KNN_tfidf = KNeighborsClassifier(n_neighbors=4)
KNN_tfidf.fit(X_train_tfidf, y_train)  

y_predict_KNN = KNN_tfidf.predict(X_test_tfidf)

# # KNN predictions performance
print("KNN: \n")
print(classification_report(y_test, y_predict_KNN))

print("\n---------------------------------------------\n")

"""scores = cross_val_score(KNN_tfidf, X_train_tfidf, y_train, cv=5)
print('Cross-Validation Accuracy Scores', scores)
print("KNN mean: ", scores.mean())"""

Logistic Regression test accuracy: 

                         precision    recall  f1-score   support

Bank account or service       0.91      0.88      0.89      1599
            Credit card       0.89      0.89      0.89      2201
       Credit reporting       0.88      0.87      0.88      2463
        Debt collection       0.86      0.88      0.87      2520
               Mortgage       0.94      0.95      0.94      2467

               accuracy                           0.89     11250
              macro avg       0.89      0.89      0.89     11250
           weighted avg       0.89      0.89      0.89     11250


---------------------------------------------


---------------------------------------------

SVM Test accuracy: 

                         precision    recall  f1-score   support

Bank account or service       0.93      0.91      0.92      1599
            Credit card       0.91      0.90      0.91      2201
       Credit reporting       0.88      0.89      0.88      24

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Gaussian Naive Bayes: 

                         precision    recall  f1-score   support

Bank account or service       0.58      0.78      0.67      1599
            Credit card       0.69      0.60      0.64      2201
       Credit reporting       0.59      0.82      0.69      2463
        Debt collection       0.75      0.49      0.59      2520
               Mortgage       0.88      0.75      0.81      2467

               accuracy                           0.68     11250
              macro avg       0.70      0.69      0.68     11250
           weighted avg       0.71      0.68      0.68     11250


---------------------------------------------

KNN: 

                         precision    recall  f1-score   support

Bank account or service       0.74      0.81      0.77      1599
            Credit card       0.76      0.76      0.76      2201
       Credit reporting       0.67      0.87      0.76      2463
        Debt collection       0.83      0.64      0.72      2520
       

'scores = cross_val_score(KNN_tfidf, X_train_tfidf, y_train, cv=5)\nprint(\'Cross-Validation Accuracy Scores\', scores)\nprint("KNN mean: ", scores.mean())'

**Deep Learning (LSTM)**

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout


# deep learning 1

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

x = tokenizer.texts_to_sequences(x.values)
x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x.shape)

y = pd.get_dummies(y).values
print('Shape of label tensor:', y.shape)

#############################################################################################

# split train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

#############################################################################################

# deep learning 2 

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

epochs = 8
batch_size = 256

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_los', patience=3)])

accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

y_test_predicted = model.predict(X_test)

print(classification_report(y_test, y_test_predicted.round(),target_names= dataset_upsampled_downsampled["product"].unique()))

